In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk

In [2]:
def cluster_cohesion(embeddings, labels):
    scores = []
    for c in np.unique(labels):
        idx = labels == c
        if idx.sum() < 2:
            continue
        sims = cosine_similarity(embeddings[idx])
        scores.append(sims.mean())
    return float(np.mean(scores))

In [3]:
# Helper to find words from the clusters
def get_top_keywords(texts, n=8):
    vec = CountVectorizer(stop_words='english').fit(texts)
    bag_of_words = vec.transform(texts)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    return sorted(words_freq, key=lambda x: x[1], reverse=True)[:n]

## Filter/replace words

In [4]:
# Download resources (run once)
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

def get_top_keywords_filtered(texts, n=10):
    # 1. SETUP LISTS
    custom_stops = list(nltk.corpus.stopwords.words('english'))
    pop_slang = [
        "lil", "gon", "bout", "em", "ayy", "uh", "huh", "ha", "vi", "doo", "wee",
        "yeah", "oh", "baby", "know", "like", "got", "just", "don", "ve", "ll", 
        "want", "need", "love", "time", "way", "make", "say", "come", "go", "right",
        "look", "good", "feel", "really", "cause", "wanna", "gonna", "gotta", "ain",
        "girl", "boy", "man", "woman", "hey", "ooh", "whoa", "shit", "fuck", "bitch",
        "nigga", "niggas", "damn", "ass", "tell", "think", "never", "back", "let",
        "swag", "yuh", "hum", "who", "what", "where", "why", "top", "call", "put",
        "gang", "thug", "bro", "pussy", "tryna", "chick", "girls", "slatt", "mmh"
    ]
    
    abstract_concepts = [
        "life", "day", "night", "heart", "mind", "world", "everything", "nothing", 
        "things", "nothin", "songs", "song", "name", "eyes", "face", "voice", 
        "head", "hand", "hands", "god", "soul", "mind", "pain", "hope", "wish", 
        "fame", "lie", "lies", "truth", "word", "words", "end", "reason", "part",
        "told", "saw", "knew", "met", "said", "made", "found", "came", "went",
        "die", "born", "live", "dead", "death", "control", "move", "wait", "hold",
        "stop", "start", "change", "keep", "leave", "stay", "believe", "remember"
    ]
    
    NAME_REPLACEMENTS = {
        "regina": "queen", "veronica": "girl", "heather": "student",
        "hamilton": "soldier", "burr": "man", "jefferson": "politician",
        "elphaba": "witch", "glinda": "friend", "evan": "boy",
        "connor": "friend", "usnavi": "guy", "vanessa": "girl"
    }

    # 2. APPLY NAME REPLACEMENTS (The Fix)
    processed_texts = []
    for t in texts:
        t_lower = t.lower() # Lowercase first
        for name, replacement in NAME_REPLACEMENTS.items():
            t_lower = t_lower.replace(name, replacement)
        processed_texts.append(t_lower)

    # 3. VECTORIZE (Use the processed list!)
    all_stops = custom_stops + pop_slang + abstract_concepts
    
    try:
        vec = CountVectorizer(stop_words=all_stops).fit(processed_texts)
        bag_of_words = vec.transform(processed_texts)
        sum_words = bag_of_words.sum(axis=0) 
        words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
        
        # 4. FINAL CLEANUP (Noun Check)
        cleaned_list = []
        for word, freq in sorted(words_freq, key=lambda x: x[1], reverse=True):
            if len(word) <= 2: continue 
            
            # Check Noun (NN, NNS)
            # We wrap it in list [] because pos_tag expects a list of tokens
            pos_tag = nltk.pos_tag([word])[0][1]
            if pos_tag.startswith('NN'):
                cleaned_list.append((word, freq))
                
        return cleaned_list[:n]
        
    except ValueError:
        # Handles empty clusters
        return []

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Bay_Techatham/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
data = pd.read_csv("../output/songs_with_lyrics_cleaned.csv")
lyrics1 = data['lyrics'].dropna()
    
musical_data = pd.read_csv("../dataset/musical.csv")
lyrics2 = musical_data['Lyrics'].dropna()

# --- FIX 1: Use ignore_index=True to fix the numbering ---
separated_lyrics_series = pd.concat([lyrics1, lyrics2], ignore_index=True)
    
# --- FIX 2: Convert to a Python List for the model ---
# This prevents the KeyError: 480 completely
separated_lyrics = separated_lyrics_series.tolist()

### Execution date:

In [6]:
date = '12feb'

In [7]:
import umap
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans


NUM_CLUSTERS = 78 

# A. Embed
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(separated_lyrics)

reducer = umap.UMAP(
    n_neighbors=15, 
    n_components=5, # 5 dimensions is usually safer than 4 for complex mixes
    min_dist=0.0,
    metric='cosine', 
    random_state=42
)
reduced_embeddings = reducer.fit_transform(embeddings)

# B. Cluster
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
kmeans.fit(reduced_embeddings)
cluster_labels = kmeans.labels_

/Users/Bay_Techatham/anaconda3/envs/japanInternship/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [10]:
# ==========================================
# 4. RESULTS (THE "SHOPPING LIST")
# ==========================================
print("\n" + "="*40)
print("   YOUR VISUAL SHOPPING LIST")
print("="*40)

df = pd.DataFrame({'text': separated_lyrics, 'cluster': cluster_labels})

for i in range(NUM_CLUSTERS):
    cluster_text = df[df['cluster'] == i]['text'].tolist()
    
    # Use the new function
    keywords = get_top_keywords_filtered(cluster_text, n=20)
    
    print(f"\n📁 CLUSTER {i+1} (Sample: {len(cluster_text)})")
    # Print just the words, not the frequency counts
    print(f"   Keywords: {[k[0] for k in keywords]}")
    print(f"   👉 DECISION: What 3D asset matches these nouns?")
    


   YOUR VISUAL SHOPPING LIST

📁 CLUSTER 1 (Sample: 14)
   Keywords: ['christmas', 'merry', 'chorus', 'verse', 'snow', 'bells', 'feliz', 'navidad', 'jingle', 'side', 'everyday', 'angel', 'sleigh', 'wop', 'mine', 'freya', 'skye', 'reindeer', 'thank', 'tonight']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 2 (Sample: 7)
   Keywords: ['spoken', 'galinda', 'friend', 'witch', 'delia', 'shiz', 'students', 'cady', 'aaron', 'lydia', 'jared', 'betelgeuse', 'sung', 'charles', 'zoe', 'daylight', 'home', 'wan', 'heidi', 'maxie']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 3 (Sample: 16)
   Keywords: ['chorus', 'verse', 'future', 'michael', 'nostalgia', 'knock', 'sound', 'bathroom', 'thing', 'hell', 'joe', 'bet', 'home', 'bridge', 'pre', 'phone', 'hate', 'bourbon', 'drinkin', 'outro']
   👉 DECISION: What 3D asset matches these nouns?

📁 CLUSTER 4 (Sample: 7)
   Keywords: ['soldier', 'washington', 'company', 'lafayette', 'laurens', 'sir', 'woah', 'buck', 'lee', 'mull

In [9]:
import pickle

model_bundle = {
    "kmeans_model": kmeans,
    "umap_reducer": reducer,  # <--- CRITICAL! Needed to shrink new data to 5D
    "sbert_model_name": 'all-MiniLM-L6-v2'
}

with open(f'../output/pickle/clustering_bundle_{ date }.pkl', 'wb') as f:
    pickle.dump(model_bundle, f)

## Manually Label

In [11]:
# 2. SAVE YOUR ASSET MAP
# Mapped to match your SBERT Keywords exactly:
# 1. Urban Environment
# 2. Room Environment (snow, grass, dark)
# 3. Ocean Environment
# 4. Forest Environment (snow, grass, boat, dark)
# all option has [morning/day/night/evening]

preset_map = {
    0: "forest_night_snow", 
    1: "urban_evening", 
    2: "urban_night",  
    3: "forest_night_grass", 
    4: "ocean_morning",
    5: "urban_night",   
    6: "urban_day",  
    7: "room_evening_dark",   
    8: "forest_night_boat",   
    9: "forest_day_grass",
    10: "ocean_evening",
    11: "ocean_morning",
    12: "urban_night",
    13: "ocean_morning",
    14: "forest_day_snow",
    15: "urban_day",
    16: "room_evening_dark",
    17: "room_night_dark",
    18: "urban_morning",
    19: "room_night_dark",
    20: "urban_evening",
    21: "urban_evening",
    22: "urban_evening",
    23: "room_evening_grass",
    24: "ocean_night",
    25: "forest_evening_dark",
    26: "urban_night",
    27: "room_evening_grass",
    28: "forest_day_grass",
    29: "forest_night_dark",
    30: "urban_evening",
    31: "room_night_dark",
    32: "urban_day",
    33: "forest_night_snow",
    34: "forest_night_grass",
    35: "forest_evening_boat",
    36: "urban_night",
    37: "forest_morning_grass",
    38: "forest_evening_grass",
    39: "forest_night_dark",
    40: "forest_day_grass",
    41: "room_evening_dark",
    42: "urban_night",
    43: "urban_day",
    44: "urban_night",
    45: "urban_night",
    46: "ocean_night",
    47: "urban_night",
    48: "urban_morning",
    49: "forest_night_dark",
    50: "urban_day",
    51: "forest_morning_grass",
    52: "urban_day",
    53: "forest_night_dark",
    54: "room_evening_dark",
    55: "forest_morning_snow",
    56: "urban_evening",
    57: "urban_evening",
    58: "room_evening_dark",
    59: "ocean_day",
    60: "urban_night",
    61: "urban_day",
    62: "forest_day_grass",
    63: "room_evening_grass",
    64: "forest_evening_grass",
    65: "room_day_grass",
    66: "ocean_day",
    67: "ocean_evening",
    68: "forest_grass",
    69: "urban_night",
    70: "urban_night",
    71: "urban_evening",
    72: "forest_night_dark",
    73: "urban_day",
    74: "forest_day_grass",
    75: "urban_night",
    76: "ocean_evening",
    77: "urban_night"
}

with open(f'../output/pickle/preset_labels_{ date }.pkl', 'wb') as f:
    pickle.dump(preset_map, f)
print(f"✅ Labels saved as 'preset_labels_{ date }.pkl'")

✅ Labels saved as 'preset_labels_12feb.pkl'
